In [49]:
import pandas as pd
import mysql.connector
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

%matplotlib inline

In [50]:
# !pip install pymssql pandas imbalanced-learn scikit-learn

In [52]:

# Connect to SQL Server
conn = mysql.connector.connect(host="localhost", user="root", password="root", database="GooglePlayStore")

# Fetch Data
query = "SELECT * FROM installs"
df = pd.read_sql(query, conn)

# Close connection
conn.close()

df = df.drop('id', axis=1)
# Show first few rows
print(df.head())



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22212\4068824899.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


  Category  Free  Size_in_Mb Content_Rating  Ad_Supported  In_App_Purchases  \
0        0     1        10.0              0             0                 0   
1        1     1         2.9              0             1                 0   
2        2     1         3.7              0             0                 0   
3        3     1         1.8              0             1                 0   
4        1     1         6.2              0             0                 0   

   Installs_Category  
0  Between 0 and 10k  
1  Between 0 and 10k  
2  Between 0 and 10k  
3  Between 0 and 10k  
4  Between 0 and 10k  


In [53]:
df.Installs_Category.value_counts()

Between 0 and 10k        1746195
Between 10k and 50k       247706
Between 100k and 500k     104339
Between 50k and 100k       71973
Between 1M and 5M          30558
Between 500k and 1M        25081
Between 5M and 10M          5791
Between 10M and 50M         5271
Between 50M and 100M         629
Greater than 100M            428
Name: Installs_Category, dtype: int64

## Feature Extraction

In [54]:
df.shape

(2237971, 7)

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
import pandas as pd

# Split features and target variable
X = df.drop(['Installs_Category'], axis=1)
y = df['Installs_Category']

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=7, stratify=y)

# Initialize DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(random_state=7)

# Fit the model
dt_classifier.fit(X_train, Y_train)

# Predict and evaluate
y_pred = dt_classifier.predict(X_test)

# Extract feature importances
importances = dt_classifier.feature_importances_

# Rank the features
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the ranked features
mean_imp = dt_classifier.feature_importances_.mean()
selected_features = X_train.columns[dt_classifier.feature_importances_ > mean_imp]
print("Selected Features Based on Decision Tree Importance:")
print(selected_features)

# Perform Recursive Feature Elimination (RFE)
rfe = RFE(DecisionTreeClassifier(random_state=7))
rfe.fit(X_train, Y_train)

# Print feature rankings
print("\nRFE Feature Rankings:")
for rank, feature in sorted(zip(rfe.ranking_, X_train.columns)):
    print(f"{feature}: {rank}")


Selected Features Based on Decision Tree Importance:
Index(['Category', 'Size_in_Mb'], dtype='object')

RFE Feature Rankings:
Ad_Supported: 1
Category: 1
Size_in_Mb: 1
In_App_Purchases: 2
Content_Rating: 3
Free: 4


Models


In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Target variable
y = df['Installs_Category']

# Feature Selection (Removing target and leakage features)
X = df.drop(['Installs_Category'], axis=1)

# Split dataset into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=7)

# Label Encoding for Target Variable
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

results = []  # Store results for comparison

def evaluate_model(model, X_test, Y_test_encoded):
    Y_pred_encoded = model.predict(X_test)
    accuracy = accuracy_score(Y_test_encoded, Y_pred_encoded)
    f1 = f1_score(Y_test_encoded, Y_pred_encoded, average='weighted')

    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 40)

    # Store results
    results.append({"Model": model.__class__.__name__, "Accuracy": accuracy, "F1 Score": f1})


In [58]:
# Applying Model CatBoost Model
import pickle
Cat_Boost = CatBoostClassifier(verbose = 200, n_estimators = 2500, learning_rate=0.1, depth=8,early_stopping_rounds=300)
# Cat_Boost = CatBoostClassifier(verbose = 0)
Cat_Boost.fit(X_train_scaled, Y_train_encoded)
cb_ac=Cat_Boost.score(X_train_scaled, Y_train_encoded)
print("CatBoost_Accuracy: ",cb_ac)

with open("installs_model.pkl", "wb") as f:
    pickle.dump(Cat_Boost, f)

0:	learn: 1.7939374	total: 1.4s	remaining: 58m 20s
200:	learn: 0.7594271	total: 4m 32s	remaining: 51m 54s
400:	learn: 0.7552239	total: 9m 3s	remaining: 47m 26s
600:	learn: 0.7527027	total: 13m 31s	remaining: 42m 43s
800:	learn: 0.7508736	total: 18m 19s	remaining: 38m 52s
1000:	learn: 0.7492720	total: 23m 9s	remaining: 34m 40s
1200:	learn: 0.7480571	total: 27m 52s	remaining: 30m 9s
1400:	learn: 0.7468904	total: 33m 49s	remaining: 26m 31s
1600:	learn: 0.7458142	total: 38m 24s	remaining: 21m 33s
1800:	learn: 0.7448200	total: 43m 8s	remaining: 16m 44s
2000:	learn: 0.7438664	total: 47m 48s	remaining: 11m 55s
2200:	learn: 0.7430394	total: 52m 27s	remaining: 7m 7s
2400:	learn: 0.7422485	total: 57m 5s	remaining: 2m 21s
2499:	learn: 0.7418744	total: 59m 17s	remaining: 0us
CatBoost_Accuracy:  0.7825523002670149


In [ ]:
#Applying Model Logistic Regression


log_reg = LogisticRegression(max_iter=500, class_weight='balanced')
log_reg.fit(X_train_scaled, Y_train_encoded)
evaluate_model(log_reg, X_test_scaled, Y_test_encoded)


Model: LogisticRegression
Accuracy: 0.4403
F1 Score: 0.5330
----------------------------------------


In [ ]:
#Applying Model RandomForest

rf_clf = RandomForestClassifier(n_estimators=100, random_state=7, class_weight='balanced')
rf_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(rf_clf, X_test_scaled, Y_test_encoded)


Model: RandomForestClassifier
Accuracy: 0.4188
F1 Score: 0.5202
----------------------------------------


In [ ]:
#Applying Model DecisionTree Classifier

dt_clf = DecisionTreeClassifier(random_state=7)
dt_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(dt_clf, X_test_scaled, Y_test_encoded)


Model: DecisionTreeClassifier
Accuracy: 0.7723
F1 Score: 0.6926
----------------------------------------


In [ ]:
#Applying Model Naive Bayesian

nb_clf = GaussianNB()
nb_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(nb_clf, X_test_scaled, Y_test_encoded)

Model: GaussianNB
Accuracy: 0.6015
F1 Score: 0.6205
----------------------------------------


In [ ]:
# Applying K-Nearest Neighbors


knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(knn_clf, X_test_scaled, Y_test_encoded)


Model: KNeighborsClassifier
Accuracy: 0.7618
F1 Score: 0.6932
----------------------------------------


In [ ]:
# Fine-tuning Linear Discriminant Analysis (LDA)
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(lda_clf, X_test_scaled, Y_test_encoded)


Model: LinearDiscriminantAnalysis
Accuracy: 0.7712
F1 Score: 0.6867
----------------------------------------


In [ ]:
# Applying Gradient Boosting Classifier


gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
gb_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(gb_clf, X_test_scaled, Y_test_encoded)


Model: GradientBoostingClassifier
Accuracy: 0.7801
F1 Score: 0.6861
----------------------------------------


In [ ]:
# Applying AdaBoost Classifier

ada_clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
ada_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(ada_clf, X_test_scaled, Y_test_encoded)


Model: AdaBoostClassifier
Accuracy: 0.7798
F1 Score: 0.6834
----------------------------------------


In [ ]:
# Applying XGBoost Classifier


xgb_clf = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=7)
xgb_clf.fit(X_train_scaled, Y_train_encoded)
evaluate_model(xgb_clf, X_test_scaled, Y_test_encoded)


Model: XGBClassifier
Accuracy: 0.7804
F1 Score: 0.6869
----------------------------------------
